In [1]:
import gc
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm.notebook import tqdm
BASE = '../data/'

def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true != 0) | (y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

def vsmape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true != 0) | (y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * smap

In [2]:
# raw = pd.read_csv("../preprocess/boosting_preprocess_neighbor.csv")
raw  = pd.read_csv("../data/preprocessing.csv")
features = ['state_i',
 'mbd_lag_1',
 'act_lag_1',
 'mbd_lag_2',
 'act_lag_2',
 'mbd_lag_3',
 'act_lag_3',
 'mbd_rollmea2_1',
 'mbd_rollmea4_1',
 'mbd_rollmea6_1',
 'neighbor_average',
 'pct_bb',
 'pct_college',
 'pct_foreign_born',
 'pct_it_workers',
 'median_hh_inc']

In [3]:
# feature selection

def feature_select_pearson(train, features, top_n=10):
    """
    Use Pearson coefficient for correlation feature selection
    :param train: train_set
    :param test: test_set
    :return: train_set and test_set after the feature selection
    """
    print('feature_select...')
    featureSelect = features[:]

    # Remove features with more than 0.99 missing values
    for fea in features:
        if train[fea].isnull().sum() / train.shape[0] >= 0.99:
            featureSelect.remove(fea)

    # perform Pearson correlation calculation
    corr = []
    for fea in featureSelect:
        corr.append(abs(train[[fea, 'microbusiness_density']].fillna(0).corr().values[0][1]))

    # select the top 5 features for modeling (the selection number is flexible)
    se = pd.Series(corr, index=featureSelect).sort_values(ascending=False)
    feature_select = ['mbd_lag_1'] + se[:top_n].index.tolist()
    feature_select = list(set(feature_select)) # unique values
    print('done')
    return train.drop([item for item in features if item not in feature_select], axis=1), feature_select

In [5]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
raw['neighbor_average'] = lbl.fit_transform(raw['neighbor_average'].astype(str))# transform the wrong value into string format



In [8]:
import copy
# Select the optimal number of features for fitting
# find out the best number of features
best_n_top = 5
best_smape = 10000
for n_top in range(5, len(features)+1):
    feature_selection_set, tmp_features = copy.deepcopy(feature_select_pearson(raw, features, n_top))
    print(f"tmp features: {tmp_features}")
# ACT_THR :actual threshold
# ABS_THR: abstract threshold
    ACT_THR = 1.8
    ABS_THR = 1.00
    feature_selection_set['ypred_last'] = np.nan
    feature_selection_set['ypred'] = np.nan
    feature_selection_set['k'] = 1.
    VAL = []
    BEST_ROUNDS = []
    for TS in range(29, 32):
        print(TS)
    # add xgb.set_config to avoid early stopping malfunction
        xgb.set_config(verbosity=0)
        model = xgb.XGBRegressor(
            objective='reg:pseudohubererror',
            #objective='reg:squarederror',
            tree_method="hist",
            n_estimators=500,
            learning_rate=0.075,
            max_leaves = 17,
            subsample=0.50,
            colsample_bytree=0.50,
            max_bin=4096,
            n_jobs=2,
            eval_metric='mae',
            early_stopping_rounds=70,
        )

        train_indices = (feature_selection_set.istest==0) & (feature_selection_set.dcount  < TS) & (feature_selection_set.dcount >= 1) & (feature_selection_set.lastactive>ACT_THR)  & (feature_selection_set.lasttarget>ABS_THR) 
        valid_indices = (feature_selection_set.istest==0) & (feature_selection_set.dcount == TS)
        model.fit(
            feature_selection_set.loc[train_indices, tmp_features],
            feature_selection_set.loc[train_indices, 'target'].clip(-0.0043, 0.0045),
            eval_set=[(feature_selection_set.loc[valid_indices, tmp_features], feature_selection_set.loc[valid_indices, 'target'])],
            verbose=50,
        )
        best_rounds = model.best_iteration
        BEST_ROUNDS.append(model.best_iteration)
        ypred = model.predict(feature_selection_set.loc[valid_indices, tmp_features])
        feature_selection_set.loc[valid_indices, 'k'] = ypred + 1
        feature_selection_set.loc[valid_indices,'k'] = feature_selection_set.loc[valid_indices,'k'] * feature_selection_set.loc[valid_indices,'microbusiness_density']

        # Validate
        lastval = feature_selection_set.loc[feature_selection_set.dcount==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
        dt = feature_selection_set.loc[feature_selection_set.dcount==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']

        df = feature_selection_set.loc[feature_selection_set.dcount==(TS+1), ['cfips', 'microbusiness_density', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)
        df['pred'] = df['cfips'].map(dt)
        df['lastval'] = df['cfips'].map(lastval)

        df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
        df.loc[df['lastval']<=ABS_THR, 'pred'] = df.loc[df['lastval']<=ABS_THR, 'lastval']
        feature_selection_set.loc[feature_selection_set.dcount==(TS+1), 'ypred'] = df['pred'].values
        feature_selection_set.loc[feature_selection_set.dcount==(TS+1), 'ypred_last'] = df['lastval'].values

        print(f'TS: {TS}')
        print('Last Value SMAPE:', smape(df['microbusiness_density'], df['lastval']) )
        print('XGB SMAPE:', smape(df['microbusiness_density'], df['pred']))
    
    cur_smape = smape(df['microbusiness_density'], df['pred'])
    print(f"n_top: {n_top}")
    print(f"current smape: {cur_smape}")
    print(f"best smape: {best_smape}")
    if n_top == 5:
        best_smape = cur_smape
    elif cur_smape < best_smape:
        best_smape = cur_smape
        best_n_top = n_top
        print("best_smape updated!")
            
    del feature_selection_set
    del tmp_features

feature_select...
done
tmp features: ['pct_college', 'pct_it_workers', 'pct_bb', 'mbd_lag_1', 'pct_foreign_born', 'median_hh_inc']
29
[0]	validation_0-mae:0.45135
[50]	validation_0-mae:0.01316
[100]	validation_0-mae:0.01090
[150]	validation_0-mae:0.01091
[200]	validation_0-mae:0.01092
[250]	validation_0-mae:0.01092
[300]	validation_0-mae:0.01092
[350]	validation_0-mae:0.01092
[400]	validation_0-mae:0.01092
[450]	validation_0-mae:0.01093
[499]	validation_0-mae:0.01093
TS: 29
Last Value SMAPE: 1.0868726017655663
XGB SMAPE: 1.0897397358896264
30
[0]	validation_0-mae:0.44530
[50]	validation_0-mae:0.01204
[100]	validation_0-mae:0.01312
[150]	validation_0-mae:0.01318
[200]	validation_0-mae:0.01318
[250]	validation_0-mae:0.01318
[300]	validation_0-mae:0.01317
[350]	validation_0-mae:0.01317
[400]	validation_0-mae:0.01317
[450]	validation_0-mae:0.01318
[499]	validation_0-mae:0.01317
TS: 30
Last Value SMAPE: 1.318087470449913
XGB SMAPE: 1.304298884208811
31
[0]	validation_0-mae:0.45100
[50]	vali

In [6]:
best_smape

1.0623562196689202

In [7]:
raw, features = feature_select_pearson(raw, features, best_n_top)

feature_select...
done


In [8]:
raw

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,year,month,...,act_lag_2,act_lag_3,mbd_rollmea4_1,mbd_rollmea6_1,neighbor_average,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,2.856021,1249.0,0,2019,8,...,NaN,NaN,NaN,NaN,2157.8,80,16,2,0,58731
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0,2019,9,...,NaN,NaN,0.010101,0.010101,2161.6,80,16,2,0,58731
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0,2019,10,...,20.0,NaN,0.069366,0.069366,2158.4,80,16,2,0,58731
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0,2019,11,...,45.0,-6.0,0.048878,0.048878,2150.0,80,16,2,0,58731
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0,2019,12,...,-26.0,45.0,0.048878,0.048878,2215.4,80,16,2,0,58731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147340,56045_2023-02-01,56045,Weston County,Wyoming,2023-02-01,NaN,NaN,1,2023,2,...,NaN,NaN,NaN,0.000000,NaN,81,13,1,0,65566
147341,56045_2023-03-01,56045,Weston County,Wyoming,2023-03-01,NaN,NaN,1,2023,3,...,NaN,NaN,NaN,0.000000,NaN,81,13,1,0,65566
147342,56045_2023-04-01,56045,Weston County,Wyoming,2023-04-01,NaN,NaN,1,2023,4,...,NaN,NaN,NaN,NaN,NaN,81,13,1,0,65566
147343,56045_2023-05-01,56045,Weston County,Wyoming,2023-05-01,NaN,NaN,1,2023,5,...,NaN,NaN,NaN,NaN,NaN,81,13,1,0,65566


In [9]:
features

['pct_college',
 'act_lag_3',
 'pct_bb',
 'mbd_rollmea6_1',
 'pct_foreign_born',
 'median_hh_inc',
 'pct_it_workers',
 'act_lag_2',
 'act_lag_1',
 'mbd_rollmea4_1',
 'mbd_lag_1',
 'neighbor_average']

In [10]:
ACT_THR = 1.8
ABS_THR = 1.00
raw['ypred_last'] = np.nan
raw['ypred'] = np.nan
raw['k'] = 1.
VAL = []
BEST_ROUNDS = []
for TS in range(29, 38):
    print(TS)
    
    model = xgb.XGBRegressor(
        objective='reg:pseudohubererror',
        #objective='reg:squarederror',
        tree_method="hist",
        n_estimators=4999,
        learning_rate=0.0075,
        max_leaves = 17,
        subsample=0.50,
        colsample_bytree=0.50,
        max_bin=4096,
        n_jobs=2,
        eval_metric='mae',
        early_stopping_rounds=70,
    )
            
    train_indices = (raw.istest==0) & (raw.dcount  < TS) & (raw.dcount >= 1) & (raw.lastactive>ACT_THR)  & (raw.lasttarget>ABS_THR) 
    valid_indices = (raw.istest==0) & (raw.dcount == TS)
    model.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, 'target'].clip(-0.0043, 0.0045),
        eval_set=[(raw.loc[valid_indices, features], raw.loc[valid_indices, 'target'])],
        verbose=500,
    )
    best_rounds = model.best_iteration
    BEST_ROUNDS.append(model.best_iteration)
    ypred = model.predict(raw.loc[valid_indices, features])
    raw.loc[valid_indices, 'k'] = ypred + 1
    raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'microbusiness_density']

    # Validate
    lastval = raw.loc[raw.dcount==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
    dt = raw.loc[raw.dcount==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']
    
    df = raw.loc[raw.dcount==(TS+1), ['cfips', 'microbusiness_density', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)
    df['pred'] = df['cfips'].map(dt)
    df['lastval'] = df['cfips'].map(lastval)
    
    df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
    df.loc[df['lastval']<=ABS_THR, 'pred'] = df.loc[df['lastval']<=ABS_THR, 'lastval']
    raw.loc[raw.dcount==(TS+1), 'ypred'] = df['pred'].values
    raw.loc[raw.dcount==(TS+1), 'ypred_last'] = df['lastval'].values
    
    print(f'TS: {TS}')
    print('Last Value SMAPE:', smape(df['microbusiness_density'], df['lastval']) )
    print('XGB SMAPE:', smape(df['microbusiness_density'], df['pred']))
    print()


ind = (raw.dcount>=30)&(raw.dcount<=38)
print( 'XGB SMAPE:', smape( raw.loc[ind, 'microbusiness_density'],  raw.loc[ind, 'ypred'] ) )
print( 'Last Value SMAPE:', smape( raw.loc[ind, 'microbusiness_density'],  raw.loc[ind, 'ypred_last'] ) )


29
[0]	validation_0-mae:0.49351
[500]	validation_0-mae:0.01429
[897]	validation_0-mae:0.01091
TS: 29
Last Value SMAPE: 1.0868726017655663
XGB SMAPE: 1.0738047669774673

30
[0]	validation_0-mae:0.48746
[500]	validation_0-mae:0.01256
[641]	validation_0-mae:0.01206
TS: 30
Last Value SMAPE: 1.318087470449913
XGB SMAPE: 1.1474297430595695

31
[0]	validation_0-mae:0.49315
[500]	validation_0-mae:0.01466
[923]	validation_0-mae:0.01118
TS: 31
Last Value SMAPE: 1.1258309832479911
XGB SMAPE: 1.1055844960714658

32
[0]	validation_0-mae:0.49995
[500]	validation_0-mae:0.01702
[1000]	validation_0-mae:0.00928
[1500]	validation_0-mae:0.00913
[1897]	validation_0-mae:0.00913
TS: 32
Last Value SMAPE: 0.897969439640235
XGB SMAPE: 0.9101787950611636

33
[0]	validation_0-mae:0.48932
[500]	validation_0-mae:0.01450
[681]	validation_0-mae:0.01329
TS: 33
Last Value SMAPE: 1.3686285670946152
XGB SMAPE: 1.2613709283532193

34
[0]	validation_0-mae:0.47751
[500]	validation_0-mae:0.01611
[509]	validation_0-mae:0.0163

In [11]:
# Prediction
raw['error'] = vsmape(raw['microbusiness_density'], raw['ypred'])
raw['error_last'] = vsmape(raw['microbusiness_density'], raw['ypred_last'])
raw.loc[(raw.dcount==30), ['microbusiness_density', 'ypred', 'error', 'error_last'] ]

,microbusiness_density,ypred,error,error_last
30,3.334431,3.300729,1.015863,1.135557
77,7.823300,7.738336,1.091972,1.155810
124,1.206827,1.187959,1.575711,1.687769
171,1.236650,1.215709,1.707845,1.834867
218,1.777708,1.754112,1.336190,1.403959
...,...,...,...,...
147140,2.892446,2.926768,1.179620,1.179620
147187,25.438322,25.551003,0.441979,0.368550
147234,3.954258,3.757571,5.100897,5.183206
147281,3.027295,3.027295,0.000000,0.000000
